# 2. Databricks Managed MCP Server を用いたエージェント実装（ベータ機能）

## 概要
- **MCP (Model Context Protocol)**: AIモデルが外部ツールと通信するための標準プロトコル
- **Databricks Managed MCP Server**: Databricksが提供するマネージドなMCPサーバー
- **期待される利点**:
  - 標準化されたツールアクセス
  - セキュアな認証・権限管理
  - 複数サービスへの統一的なアクセス

In [0]:
%pip install -U -qqqq mcp>=1.9 databricks-sdk[openai] databricks-agents>=1.0.0 databricks-mcp databricks-langchain uv langgraph==0.3.4
%restart_python

In [0]:
import asyncio
from databricks.sdk import WorkspaceClient
from databricks_mcp import DatabricksMCPClient
import nest_asyncio
nest_asyncio.apply()

workspace_client = WorkspaceClient()
host = workspace_client.config.host
genie_space_id = '***'
MANAGED_MCP_SERVER_URLS = [
    f"{host}/api/2.0/mcp/genie/{genie_space_id}",
]

async def get_tools():
  results = []
  for server_url in MANAGED_MCP_SERVER_URLS:
    mcp_client = DatabricksMCPClient(server_url=server_url, workspace_client=workspace_client)
    tools = await mcp_client.list_tools()
    results.append(tools)
  return results

mcp_tools = asyncio.run(get_tools())

print(mcp_tools)

## まとめ

### 結果
Databricks Free Edition（us-east-2）では **Databricks Managed MCP Servers は利用できません**でした。><

エラーメッセージ：
```
NOT_FOUND: Databricks managed MCP servers are not enabled. Please enroll in the beta for this feature.
```

### Managed MCPの期待される利点（ベータ版参加時）
- **標準化されたツールアクセス** - MCPプロトコルによる統一的なインターフェース
- **セキュアな認証・権限管理** - Databricksのセキュリティ機能を活用
- **複数サービスへの統一的なアクセス** - 様々なツールを同じ方法で利用可能

### 代替案
- Unity Catalog Function を使用したカスタムツール実装（ノートブック1）
- GenieAgent を使用したマルチエージェント実装（ノートブック3）